In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import torch

In [3]:
from storage.har_datasets import HARTHDataset, UCI_HARDataset, split_by_test_subject, sts_medoids
from s3ts.api.dms.har_datasets import LDFDataset, DFDataset
from storage.label_mappings import *
from s3ts.api.nets.methods import create_model_from_DM, train_model, test_model

In [4]:
from torchvision.transforms import Normalize

In [5]:
UCI_HAR_LABELS

{1: 'WALKING',
 2: 'WALKING_UPSTAIRS',
 3: 'WALKING_DOWNSTAIRS',
 4: 'SITTING',
 5: 'STANDING',
 6: 'LAYING'}

In [6]:
label_mapping = np.zeros(7)
label_mapping[1:] = np.arange(6)

ds = UCI_HARDataset("./datasets/UCI-HAR/", split="both", wsize=64, normalize=True, label_mapping=label_mapping)
len(ds)

657344

In [7]:
if not os.path.exists("./datasets/UCI-HAR/meds.npz"):
    meds = sts_medoids(ds, n=500)
    with open("./datasets/UCI-HAR/meds.npz", "wb") as f:
        np.save(f, meds)
else:
    meds = np.load("./datasets/UCI-HAR/meds.npz")

dfds = DFDataset(ds, patterns=meds, w=0.1, dm_transform=None, ram=False)

Loading cached dissimilarity frames if available...


In [8]:
DM = []

np.random.seed(42)
for i in np.random.choice(np.arange(len(dfds)), 500):
    dm, _, _ = dfds[i]
    DM.append(dm)

DM = torch.stack(DM)

dm_transform = Normalize(mean=DM.mean(dim=[0, 2, 3]), std=DM.std(dim=[0, 2, 3]))

In [9]:
dfds.dm_transform = dm_transform

In [10]:
data_split = split_by_test_subject(ds, [21, 22, 23, 24, 25, 26, 27, 28, 29])

dm = LDFDataset(dfds, data_split=data_split, batch_size=64, random_seed=42, num_workers=8)

In [11]:
print(len(dm.ds_test))
print(len(dm.ds_train))

188096
469248


In [12]:
model = create_model_from_DM(dm, name=None, 
        dsrc="img", arch="cnn", task="cls")

Input shape:  torch.Size([1, 6, 64, 64])
Latent shape:  torch.Size([1, 40, 4, 64])


In [ ]:
model, data = train_model(dm, model, max_epochs=2)
print(data)